<div class="jupyter-biolm-header">
    <img style="float: left; padding-right: 10px; height: 60px" src="https://d31e6ufxekikrt.cloudfront.net/static/ui/images/logo.png">
    <p>
    <br>
    <br>
    <br>
    </p>
</div>

## ProGen2-OAS Antibody Sequence and Structure Generation Tutorial (Utomilumab)

In this tutorial, we demonstrates the use of the BioLM.ai API with the ProGen2-OAS model to generate a library and predict the structure of Utomilumab antibody sequences that are annotated and numbered, have Levenshtein distances calculated, and with new CDR3 region sequences. 


<br>

<table class="jupyter-biolm-header-table" style="width: 100%; border-collapse: collapse; background-color: white; float: left;">
    <tr>
        <td style="text-align: left; vertical-align: middle; background-color: white;">
            <img src="https://www.svgrepo.com/show/354202/postman-icon.svg" style="height: 15px; float: left; padding-right: 10px;"><a href="https://api.biolm.ai/#2424fa3a-144c-4d8e-8391-9e69b9df15d0">  <h5 style="margin: 0;"><b>Postman API Docs</b></h5></a>
        </td>
        <td style="text-align: left; vertical-align: middle; background-color: white;">
            <img src="https://upload.wikimedia.org/wikipedia/commons/thumb/c/c3/Python-logo-notext.svg/1869px-Python-logo-notext.svg.png" style="height: 15px; float: left; padding-right: 10px;"><a href="https://docs.biolm.ai/en/latest/model-docs/progen2/ProGen2_API.html"><h5 style="margin: 0;"><b>Python SDK Docs</b></h5></a>
        </td>
        <td style="text-align: left; vertical-align: middle; background-color: white;">
            <img src="https://github.githubassets.com/assets/GitHub-Mark-ea2971cee799.png" style="height: 15px; float: left; padding-right: 10px;"><a href="https://github.com/salesforce/progen/tree/main/progen2"><h5 style="margin: 0;"><b>Github Link</b></h5></a>
        </td>
        <td style="text-align: left; vertical-align: middle; background-color: white;">
            <img src="https://info.arxiv.org/brand/images/brand-logomark-primary-large.jpg" style="height: 15px; float: left; padding-right: 10px;"><a href="https://arxiv.org/pdf/2206.13517.pdf"><h5 style="margin: 0;"><b>Paper Link</b></h5></a>
        </td>
    </tr>
</table>

<br>

---

# Utomilumab: Our Antibody of Interest.
Utomilumab is a is a fully human IgG2 monoclonal antibody that targets the CD137 receptor (also known as 4-1BB), which is a member of the tumor necrosis factor (TNF) receptor family. CD137 is a co-stimulatory molecule expressed on the surface of activated T cells and other immune cells. The engagement of CD137 with its ligand or agonistic antibodies like utomilumab can enhance T cell proliferation, cytokine production, and survival, leading to an amplified immune response. You can learn more about its mechanism of action 
<u>[here](https://cdn.pfizer.com/pfizercom/news/asco/Utomilumab_Fact_Sheet_23MAY2018.PDF)<u>



## Set your API Token

To use the BioLM API, you need an API token. You can get one from the <u>[User API Tokens](https://biolm.ai/ui/accounts/user-api-tokens/)<u> page.

Paste the API token in the cell below, as the value of the variable BIOLMAI_TOKEN.

In [3]:
BIOLMAI_TOKEN = " " # !!! YOUR API TOKEN HERE !!!

## API Call with Python Requests

In [4]:
#The below is needed if running this notebook in a Pyodide environment:

try:
    # Install packages to make API requests in JuputerLite
    import micropip
    await micropip.install('requests')
    await micropip.install('pandas')
    await micropip.install('numpy')
    await micropip.install('pyodide-http')
    # Patch requests for in-browser support
    import pyodide_http
    pyodide_http.patch_all()
except ModuleNotFoundError:
    pass  # Won't be using micropip outside of JLite
    
import requests
from IPython.display import JSON  # Helpful UI for JSON display

# Note: sadie-antibody and Levenshtein will not install in a Pyodide environment. They can be installed in a local Jupyter notebook or terminal using "pip install". 


# Import Necessary Packages and Dependencies



In [5]:
import warnings
warnings.filterwarnings('ignore', category=FutureWarning, module='')

import pandas as pd
import requests
import json
import zipfile
import Levenshtein as lev
import os
from sadie.renumbering import Renumbering
from sadie.renumbering.result import NumberingResults
from IPython.display import JSON 

## Initializing Constants, File Paths, and Sequence Data

The Utomilumab sequnce can be obtained from <u>[Thera-SAbDab](https://opig.stats.ox.ac.uk/webapps/sabdab-sabpred/therasabdab/therasummary/?INN=utomilumab&format=Whole+mAb&clintrial=Phase-II&status=Active&target=TNFRSF9/CD137/4-1BB&yearprop=2016&isotype=G2&heavy1=EVQLVQSGAEVKKPGESLRISCKGSGYSFSTYWISWVRQMPGKGLEWMGKIYPGDSYTNYSPSFQGQVTISADKSISTAYLQWSSLKASDTAMYYCARGYGIFDYWGQGTLVTVSS&light1=SYELTQPPSVSVSPGQTASITCSGDNIGDQYAHWYQQKPGQSPVLVIYQDKNRPSGIPERFSGSNSGNTATLTISGTQAMDEADYYCATYTGFGSLAVFGGGTKLTVL&heavy2=na&light2=na&struc100=6a3w:GH:DE:AB:JK/6mi2:DE:AB&struc99=None&struc95to98=None&yearrec=2017&companies=Dana-Farber+Cancer+Institute;Immatics+US;Kite+Pharma;Kyowa+Hakko+Kirin;M.+D.+Anderson+Cancer+Center;Pfizer&cond_approved=na&cond_active=Breast+cancer;Oropharyngeal+cancer;Solid+tumours;B-cell+lymphoma;Diffuse+large+B+cell+lymphoma;Follicular+lymphoma;Ovarian+cancer&cond_disc=Non-Hodgkin%27s+lymphoma;Colorectal+cancer&dev_tech=MorphoSys+HuCAL+Phage+Display&notes=)<u>


In [9]:
# Constants
json_result_file = 'ProGen2_OAS_sequences.json'
csv_file_path = 'ProGen2_OAS_sequences.csv'
json_directory = "json_results"
zip_file_path = "json_results.zip"

# The process begins by including the wild-type (wt) heavy chain sequence and a truncated version up to (and including) the fwr3 region.

wt_sequence = "EVQLVQSGAEVKKPGESLRISCKGSGYSFSTYWISWVRQMPGKGLEWMGKIYPGDSYTNYSPSFQGQVTISADKSISTAYLQWSSLKASDTAMYYCARGYGIFDYWGQGTLVTVSS" #Wild-type (wt) heavy chain sequence. length = 116 amino acids. 
wt_until_fwr3 = "EVQLVQSGAEVKKPGESLRISCKGSGYSFSTYWISWVRQMPGKGLEWMGKIYPGDSYTNYSPSFQGQVTISADKSISTAYLQWSSL" #Wild-type (wt) heavy chain sequence up to and uncluding fwr3. length= 86 amino acids (can be customized in Function 1).

print("Move on to Functions 1 and 2")

Move on to Functions 1 and 2


## Function 1 and Function 2

The below few code blocks are going to run Function 1 and Function 2. Once these two code blocks are run, the user can call both functions a few code blocks below under the title "Calling Function 1 and 2", to generate the sequence csv file that has been annotated and numbered, with Levenshtein distance calculated. 

## Function 1: Generating Sequences using ProGen2-OAS via the BioLM API

Sequence Generation: The "generate_progen2_sequences" function below sends requests to the ProGen2-OAS API to generate new antibody sequences based on the provided wild-type sequence. The function allows for various parameters such as num_requests (to control the #API requests sent), temperature, max_length, and top_p to control the generation process.
For more information on these parameters, refer the BioLM.ai <u>[API docs](https://docs.biolm.ai/en/latest/model-docs/progen2/ProGen2_API.html)<u>

The responses are saved as JSON files. The resulting sequences and their JSON files are then zipped for storage.


In [10]:
# Function 1:

def generate_progen2_sequences(wt_sequence, truncate_length=None, t=0.8, top_p=0.6, max_length=135, num_requests=2, sequences_per_request=3):  
    url = "https://biolm.ai/api/v2/progen2-oas/generate/"
    headers = {'Content-Type': 'application/json', "Authorization": f"Token {BIOLMAI_TOKEN.strip()}"}
    os.makedirs(json_directory, exist_ok=True)

    if truncate_length is not None:
        wt_until_fwr3 = wt_sequence[:truncate_length]

    all_sequences = []             # Initializing the generated sequences list.
    for i in range(num_requests):  # Iterating over the number of API requests to make.
        payload = json.dumps({     # Preparing the payload for the POST request as a JSON string
            "params": {"temperature": t, "top_p": top_p, "num_samples": sequences_per_request, "max_length": max_length},
            "items": [{"context": wt_until_fwr3}]
        })
        response = requests.post(url, headers=headers, data=payload) #response to the POST request is stored in a variable called 'response'. Note, 'response' is an object here. 

        if response.status_code == 200: #If response = 200, a successful request has been made.
            data = response.json()
            json_file_path = os.path.join(json_directory, f'json_result_{i + 1}.json')
            with open(json_file_path, 'w') as f:
                json.dump(data, f, indent=4)
            print(f"JSON results saved to {json_file_path}")

             
            # The 'results' is ackey in a dictionary. That dictionary contains a list and another list within that list.
            # The inner-most list contains three dictionaries, whose keys are 'll_sum','ll_mean' and 'sequence', and each key contains associated values.
            # We will filter sequences with 'll_mean' greater than -2 and return them. 
            filtered_sequences = filtered_sequences = [(item["sequence"], item["ll_mean"]) for item in data["results"][0] if item['ll_mean'] > -2]  

            all_sequences.extend(filtered_sequences)
        else:
            print(f"Error during generation: {response.text}")
    
    with zipfile.ZipFile(zip_file_path, 'w') as zipf:  # Creating a ZIP file to compress all the JSON files containing the results from the API requests.
        for root, _, files in os.walk(json_directory):
            for file in files:
                zipf.write(os.path.join(root, file), arcname=file)
    print(f"JSON results are zipped in {zip_file_path}")
    print ("Completed Function 1")

    return all_sequences # We now have a list of ProGen2-OAS generated sequences generated via the BioLM.ai API that have been filtered based on the 'll_mean' value



## Function 2: Sequence Annotation and Distance Calculation

The "annotate_and_calculate_distances" function below uses the SADIE renumbering API to annotate both the wild-type sequence and the generated sequences. The below function also calculates the Levenshtein distance between each generated sequence and the wild-type sequence to assess their similarity. The results, including the sequences, their annotations, and distances, are saved in a CSV file for further analysis (such as structure prediction). 

<i> You can learn more about Sadie-Antibody (Sequencing Analysis and Data library for Immunoinformatics Exploration) <u>[here](https://pypi.org/project/sadie-antibody/)<u><i>


In [11]:
#Function 2:

def annotate_and_calculate_distances(sequences):
    renumbering_api = Renumbering(scheme="imgt", region_assign="imgt", run_multiproc=True)  # creating an instance of the Renumbering class from the SADIE-Antibody library. We use the default setting of "imgt" numbering scheme and the "imgt" region assignment.
    results = []  # Initializing empty list for results. Note: we are using this list to collect dictionaries.

    wt_numbering_result = renumbering_api.run_single("wt_sequence", wt_sequence)
    wt_result_dict = {key: value[0] if isinstance(value, dict) else value for key, value in wt_numbering_result.to_dict().items()}  # Flattening the dictionary. By flattening any nested dictionaries, the values are more straightforward to work with when creating a CSV file. Note: a dictionary is relevant if we are modifying the result or adding additional information to it.
    wt_result_dict["Levenshtein_distance"] = None  # Indicating no distance calculation needed for Wild-type sequence.
    wt_result_dict['Id'] = 'wt_sequence'           # Label the row as "wt_sequence" (ie, row for the resulting csv file).
    results.append(wt_result_dict)                 # Adding this dictionary to the results list.  

    for i, (sequence, ll_mean_value) in enumerate(sequences):                                   #Iterating over the sequence list. 
        numbering_result = renumbering_api.run_single(f"generated_sequence_{i + 1}", sequence)  # 'run_single' method of the renumbering API being applied, Use of 'i+1' and f-string to make the generated sequence list more 'human-readable': ie, 'row will be named 'generated_sequence_1' instead of generated_sequence_0'
        result_dict = {key: value[0] if isinstance(value, dict) else value for key, value in numbering_result.to_dict().items()} # Flatten the dictionary. Here, we are iterating over each key-value pair in this dictionary. The code takes the first element of a value if it is a dictionary (to flatten it) or keeps the value unchanged if it is not a dictionary.
        result_dict["Levenshtein_distance"] = lev.distance(wt_sequence, sequence) 
        result_dict['Id'] = f'generated_sequence_{i + 1}' # Label the row with the sequence number. Ie, 'generated_sequence_1', 'generated_sequence_2 'and so on...
        result_dict["ll_mean"] = ll_mean_value
        results.append(result_dict) 

    results_df = pd.DataFrame(results)  # Creating a DataFrame from the results list. Each dictionary in the list becomes a row in the DataFrame.
    results_df.to_csv(csv_file_path, index=False)
    print(f"Results saved to {csv_file_path}")
    print ("Completed Function 2")



# Calling Function 1 and 2

In [12]:
generated_sequences = generate_progen2_sequences(wt_sequence, truncate_length=86) 
if generated_sequences:
    annotate_and_calculate_distances(generated_sequences)

JSON results saved to json_results/json_result_1.json
JSON results saved to json_results/json_result_2.json
JSON results are zipped in json_results.zip
Completed Function 1
Results saved to ProGen2_OAS_sequences.csv
Completed Function 2


## Examing the Generated CSV File

In [20]:
# Load the CSV file into a DataFrame
df = pd.read_csv('ProGen2_OAS_sequences.csv')

# Display the first few rows of the DataFrame
df.head(10)


,Id,sequence,domain_no,hmm_species,chain_type,e-value,score,seqstart_index,seqend_index,identity_species,...,fwr3_aa_gaps,fwr3_aa_no_gaps,cdr3_aa_gaps,cdr3_aa_no_gaps,fwr4_aa_gaps,fwr4_aa_no_gaps,leader,follow,Levenshtein_distance,ll_mean
0,wt_sequence,EVQLVQSGAEVKKPGESLRISCKGSGYSFSTYWISWVRQMPGKGLE...,0,human,H,0.0,185.000,0,115,human,...,NYSPSFQ-GQVTISADKSISTAYLQWSSLKASDTAMYYC,NYSPSFQGQVTISADKSISTAYLQWSSLKASDTAMYYC,ARGYG----IFDY,ARGYGIFDY,WGQGTLVTVSS,WGQGTLVTVSS,NaN,NaN,NaN,NaN
1,generated_sequence_1,EVQLVQSGAEVKKPGESLRISCKGSGYSFSTYWISWVRQMPGKGLE...,0,human,H,0.0,183.375,0,117,human,...,NYSPSFQ-GQVTISADKSISTAYLQWSSLKASDTAMYYC,NYSPSFQGQVTISADKSISTAYLQWSSLKASDTAMYYC,ARRGYS--YGYDY,ARRGYSYGYDY,WGQGTLVTVSS,WGQGTLVTVSS,NaN,NaN,5.0,-0.411147
2,generated_sequence_2,EVQLVQSGAEVKKPGESLRISCKGSGYSFSTYWISWVRQMPGKGLE...,0,human,H,0.0,169.500,0,131,human,...,NYSPSFQ-GQVTISADKSISTAYLQWSSLKASDTAMYYC,NYSPSFQGQVTISADKSISTAYLQWSSLKASDTAMYYC,ARHSPRGGYYDSSGYYPYYYYGMDV,ARHSPRGGYYDSSGYYPYYYYGMDV,WGQGTTVTVSS,WGQGTTVTVSS,NaN,NaN,20.0,-0.479769
3,generated_sequence_3,EVQLVQSGAEVKKPGESLRISCKGSGYSFSTYWISWVRQMPGKGLE...,0,human,H,0.0,183.250,0,123,human,...,NYSPSFQ-GQVTISADKSISTAYLQWSSLKASDTAMYYC,NYSPSFQGQVTISADKSISTAYLQWSSLKASDTAMYYC,ARHGDYYDSSGYYPFDY,ARHGDYYDSSGYYPFDY,WGQGTLVTVSS,WGQGTLVTVSS,NaN,NaN,9.0,-0.419964
4,generated_sequence_4,EVQLVQSGAEVKKPGESLRISCKGSGYSFSTYWISWVRQMPGKGLE...,0,human,H,0.0,182.500,0,122,human,...,NYSPSFQ-GQVTISADKSISTAYLQWSSLKASDTAMYYC,NYSPSFQGQVTISADKSISTAYLQWSSLKASDTAMYYC,ARRGAGGTYYYYGMDV,ARRGAGGTYYYYGMDV,WGQGTTVTVSS,WGQGTTVTVSS,NaN,NaN,12.0,-0.452040
5,generated_sequence_5,EVQLVQSGAEVKKPGESLRISCKGSGYSFSTYWISWVRQMPGKGLE...,0,human,H,0.0,167.750,0,131,human,...,NYSPSFQ-GQVTISADKSISTAYLQWSSLKASDTAMYYC,NYSPSFQGQVTISADKSISTAYLQWSSLKASDTAMYYC,ARHRRDYYDSSGYYYGGYYYYGMDV,ARHRRDYYDSSGYYYGGYYYYGMDV,WGQGTTVTVSS,WGQGTTVTVSS,NaN,NaN,20.0,-0.456397
6,generated_sequence_6,EVQLVQSGAEVKKPGESLRISCKGSGYSFSTYWISWVRQMPGKGLE...,0,human,H,0.0,180.750,0,124,human,...,NYSPSFQ-GQVTISADKSISTAYLQWSSLKASDTAMYYC,NYSPSFQGQVTISADKSISTAYLQWSSLKASDTAMYYC,ARRGGYCSSTSCYDAFDI,ARRGGYCSSTSCYDAFDI,WGQGTMVTVSS,WGQGTMVTVSS,NaN,NaN,13.0,-0.412610


The DataFrame above contains the Wild-type sequence as well as the generated sequences. Information on species, V and J gene segments and identities, frameworks and CDR3 region sequences, Levenshtein distance and ll_mean values are all given. 

Sequences were filterd and ranked based on ll_mean. In the context of ProGen2-OAS, which has been specifically trained on a diverse set of antibody sequences from the Observed Antibody Space (OAS) database, a lower ll_mean value has many implications. The model has learned the patterns associated with functional antibodies, so a lower log-likelihood mean indicates that the generated sequence aligns well with these functional patterns. Furthermore, since the training data has been clustered to reduce redundancy and ensure diversity, a lower ll_mean suggests that the sequence is a good match to the natural variation observed in the antibody space. 

The goal is to provide easy and programmatic access to language models like ProGen2-OAS, via the BioLM API, to optimize
antibody properties including thermal stability, expression, aggregation propensity, and solubility. 

# NanoBodyBuilder 2:

NanoBodyBuilder2 can be used to produce models of nanobody. If we only have the heavy chain sequence of a conventional antibody (from humans, mice, etc.), it does not automatically become a nanobody. However, for this use-case, we can use NanoBodyBuilder2 as a bioinformatics tool to predict the structure of our ProGen2-OAS generated sequences of Utomilumab.  To learn more about NanoBodyBuilder2, click <u>[here](https://github.com/oxpig/ANARCI/issues/12)<u><i>

<i>NanoBodyBuilder 2 requires openmm, pdbfixer, anarci and ImmuneBuilder.
Information on how to install and run these packages can be found <u>[here](https://opig.stats.ox.ac.uk/webapps/sabdab-sabpred/sabpred/nanobodybuilder2/)<u>

<i>Further troubleshooting information can be found <u>[here](https://github.com/oxpig/ANARCI/issues/12)<u><i>


In [21]:
# Import necessary packages and dependencies 

import pandas as pd
from ImmuneBuilder import NanoBodyBuilder2


In [22]:
# Load the CSV file generated in Function 2 above into a DataFrame: "ProGen2_OAS_sequences.csv"

csv_file_path = 'ProGen2_OAS_sequences.csv' #CSV file generated in Function 2 above.
sequences_df = pd.read_csv(csv_file_path)

# Initializing NanoBodyBuilder2
predictor = NanoBodyBuilder2()

# Looping through each row in the DataFrame from above

for index, row in sequences_df.iterrows():    # iterrows() is a generator that iterates over the rows of the DataFrame.
    sequence_id = row['Id']                   # Accessing the value in the column named 'Id'.
    sequence = {'H': row['sequence']}         # Accessing the value in the column named 'sequence'. 

                                              
    nanobody = predictor.predict(sequence)    # Generating the nanobody/antibody structure.
    output_file = f"{sequence_id}.pdb"        # Defining the output file path using the sequence ID.
    nanobody.save(output_file)                # Saving the nanobody structure to a PDB file.

    print(f"Saved the generated nanobody structure to {output_file}") # We now have a PDB files of the Utomilumab generated sequences. 




Saved the generated nanobody structure to wt_sequence.pdb
Saved the generated nanobody structure to generated_sequence_1.pdb
Saved the generated nanobody structure to generated_sequence_2.pdb
Saved the generated nanobody structure to generated_sequence_3.pdb
Saved the generated nanobody structure to generated_sequence_4.pdb
Saved the generated nanobody structure to generated_sequence_5.pdb
Saved the generated nanobody structure to generated_sequence_6.pdb


In [23]:
#Lets view the first few lines of the pdb files:

# Specify the path to your PDB file.
pdb_file_path = 'generated_sequence_1.pdb'

# Number of lines to display from the beginning of the file.
num_lines = 10

# Open the file and read the specified number of lines.
with open(pdb_file_path, 'r') as file:
    for _ in range(num_lines):
        line = file.readline()
        print(line, end='')  # Print each line, end='' prevents adding an extra newline.


REMARK  NANOBODY STRUCTURE MODELLED USING NANOBODYBUILDER2                      
REMARK  STRUCTURE REFINED USING OPENMM 8.1.1, 2024-03-29                        
ATOM      1  N   GLU H   1      -8.448  -4.352 -18.972  1.00  1.28           N  
ATOM      2  H   GLU H   1      -7.935  -3.488 -18.852  1.00  1.28           H  
ATOM      3  H2  GLU H   1      -8.232  -4.753 -19.874  1.00  1.28           H  
ATOM      4  H3  GLU H   1      -9.433  -4.143 -18.892  1.00  1.28           H  
ATOM      5  CA  GLU H   1      -8.045  -5.278 -17.891  1.00  1.28           C  
ATOM      6  HA  GLU H   1      -8.618  -6.202 -17.973  1.00  1.28           H  
ATOM      7  C   GLU H   1      -8.356  -4.649 -16.531  1.00  1.28           C  
ATOM      8  CB  GLU H   1      -6.564  -5.621 -18.071  1.00  1.28           C  




We have demonstrated that by leveraging the BioLM.ai API and ProGen2-OAS model, researchers can rapidly generate and structurally predict a diverse library of Utomilumab antibody sequences with new CDR3 regions.

CDR3 regions are the most variable parts of the antibody and play a pivotal role in determining the specificity of the antigen binding site. Due to their immense sequence diversity, remarkable conformational flexibility that changes upon antigen binding, and limited available structural data compared to other antibody regions, modeling and predicting the structure of CDR3 loops remains a significant challenge for antibody scientists. 

This tutorial equips experts with the tools to navigate the exciting world of antibody engineering. We eagerly anticipate experts utilizing our platform to create novel antibodies with customized sequences and CDR3 regions, advancing the field through their innovative research.

## Next Steps

Check out additional tutorials at <u>[jupyter.biolm.ai](https://jupyter.biolm.ai)<u>

To explore additional models and functionality, head over to our <u>[BioLM Documentation]<u>(https://docs.biolm.ai)<u>

   
#### See more use-cases and APIs on your <u>[BioLM Console Catalog](https://biolm.ai/console/catalog/).<u>
<br>

##### BioLM hosts deep learning models and runs inference at scale. You do the science.
<br>

<table class="jupyter-biolm-header-table" style="width: 100%; border-collapse: collapse; background-color: white; float: left;">
    <tr>
        <td style="text-align: left; vertical-align: middle; background-color: white;">
            <img src="https://d31e6ufxekikrt.cloudfront.net/static/ui/images/console-overview/enzyme_engineering_icon.png"  style="height: 40px; float: left; padding-right: 10px;"> Enzyme Engineering
        </td>
        <td style="text-align: left; vertical-align: middle; background-color: white;">
            <img src="https://d31e6ufxekikrt.cloudfront.net/static/ui/images/console-overview/antibody_engineering_icon.png"  style="height: 40px; float: left; padding-right: 10px;"> Antibody Engineering
        </td>
        <td style="text-align: left; vertical-align: middle; background-color: white;">
            <img src="https://d31e6ufxekikrt.cloudfront.net/static/ui/images/console-overview/biosecurity_icon.png"  style="height: 40px; float: left; padding-right: 10px;"> Biosecurity
        </td>
    </tr>
    <tr>
        <td style="text-align: left; vertical-align: middle; background-color: white;">
            <img src="https://d31e6ufxekikrt.cloudfront.net/static/ui/images/console-overview/single_cell_genomics_icon.png"  style="height: 40px; float: left; padding-right: 10px;"> Single-Cell Genomics
        </td>
        <td style="text-align: left; vertical-align: middle; background-color: white;">
            <img src="https://d31e6ufxekikrt.cloudfront.net/static/ui/images/console-overview/dna_seq_modeling_icon.png"  style="height: 40px; float: left; padding-right: 10px;"> DNA Sequence Modelling
        </td>
        <td style="text-align: left; vertical-align: middle; background-color: white;">
            <img src="https://d31e6ufxekikrt.cloudfront.net/static/ui/images/console-overview/finetuning_icon.png"  style="height: 40px; float: left; padding-right: 10px;"> Finetuning
        </td>
    </tr>
</table>

#### [**Contact us**](https://biolm.ai/ui/contact-us/) to learn more.
    
    
Author: Zeeshan Siddiqui,   Bioinformaitican @BioLM.ai

Email ID: zsiddiqui@biolm.ai

